In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("filmler_clean.csv")

In [ ]:
df.head()

In [ ]:
clean = df["clean"]

In [ ]:
len(clean)

In [ ]:
word2idx = {}
idx2word = {}

In [ ]:
i = 0
for cell in clean:
    words = cell.split()
    for word in words:
        if word in word2idx:
            continue
        word2idx[word] = i
        idx2word[i] = word
        i += 1

In [ ]:
word2idx

In [ ]:
len(word2idx)

In [ ]:
import numpy as np

In [ ]:
film_sayisi = len(clean)
vocab_size = len(word2idx)

In [ ]:
filmler = np.zeros((film_sayisi, vocab_size))

In [ ]:
filmler.shape

In [ ]:
for i, cell in enumerate(clean):
    words = cell.split()
    for word in words:
        wordIndex = word2idx[word]
        filmler[i][wordIndex] += 1


In [ ]:
filmler[7][:10]

In [ ]:
doc_frequencies = {}

In [ ]:
# 1 - 5 dk sürebilir 
for word in word2idx:
    doc_count = 0
    for film in clean:
        if word in film:
            doc_count += 1
    
    doc_frequencies[word] = doc_count


In [ ]:
sorted_frequencies = sorted(doc_frequencies, key=doc_frequencies.get, reverse=True)

In [ ]:
for word in sorted_frequencies[:10]:
    print(word, doc_frequencies[word])

In [ ]:
for word in sorted_frequencies[150:160]:
    print(word, doc_frequencies[word])

In [ ]:
for word in sorted_frequencies[-10:]:
    print(word, doc_frequencies[word])

In [ ]:
# 1 - 5 dk sürebilir
filmler_tf_idf = np.zeros((film_sayisi, vocab_size))
film_sayisi = len(clean)

for i, film in enumerate(filmler):
    if i % 1000 == 0:
        print(f"{i} inci film işleniyor...")
    for j, tf in enumerate(film):
        idf = 0
        if tf > 0:
            idf = np.log(film_sayisi / (doc_frequencies[idx2word[j]] + 1)) + 1
        tf_idf_score = tf * idf
        filmler_tf_idf[i][j] = tf_idf_score

In [ ]:
print(filmler_tf_idf[0][:10])
print(filmler[0][:10])

In [ ]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
def film_2_str(row_index):
    film = df.iloc[row_index]
    adi = film['film adi']
    turu = film['tur']
    ozet = film['ozet ve detaylar']
    if len(ozet) > 150:
        ozet = ozet[:150] + "..."

    return f"Film adı: {adi} \nTürü: {turu} \nÖzeti: {ozet}"

In [ ]:
print(film_2_str(0))

In [ ]:
def most_similar_films(film_adi, benzer_film_sayisi):
    film_indexi = df[df["film adi"] == film_adi].index[0]
    print("Aranan Film:")
    print(film_2_str(film_indexi))
    print()
    print("* " * 30)
    print()

    benzerlikler = []
    sorgu_film = filmler_tf_idf[film_indexi]
    for vec in filmler_tf_idf:
        benzerlik = cos_sim(sorgu_film, vec)
        benzerlikler.append(benzerlik)
    
    en_benzer_film_indexleri = (- np.array(benzerlikler)).argsort()[:benzer_film_sayisi + 1]
    print("Benzer Filmler:\n")
    for i, film_idx in enumerate(en_benzer_film_indexleri[1:]):
        print(str(i+1) + ") " + film_2_str(film_idx))
        print()
        print("- " * 20)
        print()
    

In [ ]:
df.iloc[:10]["film adi"]

In [ ]:
most_similar_films("Kirpi Sonic 2", 3)